<a href="https://colab.research.google.com/github/moeinset/AD_progression_prediction/blob/main/alzheimer_progression_by_neuroimaging_and.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**ALZHEIMER PROGRESSION PREDICTION WITH NEUROIMAGING AND COGNITIVE TESTS;A MACHINE LEARNING MODELS STUDY**
Alzheimer’s disease (AD)is the driving cause of dementia in older adults. Their frequency rates are expanding at an disturbing rate each year. A treatment given at an early stage of AD is more effective, and it causes less minor harm than a treatment done at a later stage.so in this study we try machine leanring method to train a model for predict the dementia progression with just  use neuroimaging  and cognitive tests at baseline.

In this training We use AD datasets collected from the Alzheimer’s Disease Neuroimaging Initiative (ADNI).the raw dataset(Merg,all collected data) were preprocessing used python and libraries like pandas in my notebook! I removed non imaging features like bood biomarkers and .. . the first cleaned data imported and convert all columns heading to lower case for better manipulating. let take a brief look use describe() method!  

In [ ]:
#import pandas for datacleaning
import pandas as pd

#import dataset
#this dataset is previously cleaned, so it include only neuroimaging,cognitive scores and demographics features
#also missing values in DX feature were droped by pandas.dropna() command becaus this will be our Target column.
df = pd.read_csv('../input/alzheimer-just-neuroimaging-and-cogtest-baseline/first_cleaned_data.csv')
df = pd.DataFrame(df) #convert data to dataframe
df.columns = df.columns.str.lower()#convert all columns heading to lower case for better manipulating
df.describe() #firs look!

**summary data description**
as you can see we have 11195 samples that include different participants(rid), with different visit session(visitcode). features data are demographics, neuroimaging and cognitive tests. descriptive statistics are shown by describe() method up there.

**STUDY DESCRIPTION**
We try to train a model(s) that can predict the progression of participants cognitive status using very non-invasive procedures(even blood biomarkers were exclude!).

**TARGET**
to aim this we should first set the target. Our target for this study is the progression of participants dementia status from baseline. We just have diagnostic status for each visit session.CN for cognitive normal, MCI for mild cognitive impairment and Dementia for Alzheimer. We dont have target directly in data, so we need to extract this from data by compare the baseline diagnosis and last session diagnosis(at least 6 month later).we have different visit time session for participants and we consider the last one(from 6 to 186 month).

**TARGET EXTRACTION ALGORITHM**
1-assign a numeric value to visit session(visitcode)(e.g. baseline = 1, month 6 = 2 and ...).
2-filter and split dataset by first visit(minimum value) and last visit(maximum value)
3-campare the diagnosis status(dx columne):
1. if baseline is normal and still stay normal = low risk dementia progression(target = HCL)
1. if baseline is normal and progress to MCI  = risk of MCI(target = mMCI)
1. if baseline is normal and progress to DEMENTIA  = risk of AD(target = AD)
1. if baseline is MCI and still stay MCI  = stable MCI(target = sMCI)
1. if baseline is MCI and progress to DEMENTIA  = progressive MCI(target = pMCI)
1. if baseline is MCI and last vist is normal!  = data collection bias(remove data)

In [ ]:
df['viscode'].unique() 
df["viscode"].replace({"bl": "1", "m06": "2",'m12': "3", 'm24': "4", 'm18': "5", 'm36': "6", 'm48': "7", 'm60': "8", 'm72': "9",
       'm108': "10", 'm96': "11", 'm84': "12", 'm120': "13", 'm132': "14", 'm144': "15", 'm156': "16", 'm168': "17",
       'm180': "18", 'm42': "19", 'm174': "20", 'm162': "21", 'm138': "22", 'm150' : "23", 'm102': "24", 'm90': "25",
       'm114': "26", 'm78': "27", 'm126': "28", 'm66': "29", 'm54': "30", 'm30': "31", 'm186': "32"}, inplace=True)
df["rid"] = pd.to_numeric(df["rid"])
df["viscode"] = pd.to_numeric(df["viscode"])


In [ ]:
dfmin = df.loc[df.groupby('rid')['viscode'].idxmin()]
dfmin.rename(columns={'dx' : 'dx_min'}, inplace=True)
dfmin = dfmin[dfmin['dx_min'] != 'Dementia']
dfmax = df.loc[df.groupby('rid')['viscode'].idxmax()]
dfmax = dfmax[dfmax['viscode'] >= 2]
dfmax.rename(columns={'dx' : 'dx_max'}, inplace=True)
df2 = pd.merge(dfmin,dfmax[['rid','dx_max']],on='rid', how='left')

conditions = [
    (df2['dx_min'] == 'CN') & (df2['dx_max'] == 'CN'),
    (df2['dx_min'] == 'CN') & (df2['dx_max'] == 'MCI'),
    (df2['dx_min'] == 'CN') & (df2['dx_max'] == 'Dementia'),
    (df2['dx_min'] == 'MCI') & (df2['dx_max'] == 'CN'),
    (df2['dx_min'] == 'MCI') & (df2['dx_max'] == 'MCI'),
    (df2['dx_min'] == 'MCI') & (df2['dx_max'] == 'Dementia')
    ]

# create a list of the values we want to assign for each condition
values = ['HCL', 'mMCI', 'AD', '','sMCI','pMCI']

# create a new column and use np.select to assign values to it using our lists as arguments
df2['target'] = np.select(conditions, values)
df2.drop(['dx_min','dx_max'],axis = 1, inplace = True)
df2

at last we have 1969 samples. but be careful! this participants survey in different visit time. you can see them in visitcode columne of primary data frame(bl=baseline, m06=month 6 and ...). but we just need baseline datas. so we extracted them!

In [ ]:
 #because we want to use just baseline features we should filter non-baseline data
df3 = df2[df2['viscode']== 1] #extract basle line by visitcod(from bl(baseline),m06(month 6) , ...)
df4 = df3.drop(['viscode','ptid','rid','fsversion_bl','fsversion','examdate_bl'], axis = 1) #drop some non usfull! features
bl_list = df4.filter(regex='bl').columns #make a list in columns that have bl key(baseline)
non_bl_list = [s.replace("_bl", "") for s in bl_list]#make a list of columns name that used to have bl in them(basle line)
df5 = df4.drop(columns=non_bl_list) #remove the columns those arent baseline
df5 = df5[df5['target'] != '0']
df5 = df5[df5['target'] != '']

after baseline features extraction let take a look at missing values. in some data it would be better to first analysis noise and then missing values. but in this case seems noise features are not the issue.

In [ ]:
pd.set_option('display.max_rows', 100) #set display option maximum to 100
miss = df5.isnull().sum().sort_values(ascending = False) #make a list of summarised sorted missing val
miss = pd.DataFrame(miss) #convert them to dataframe
miss.columns = ['MissValues'] #change columne name
miss

as you see we have a huge number of missing values. in usual case we should contact with data set owner for better deciding. but in this case we chose to drop feauturse with the critical missing values. in orther we get help from article review to NOT remove important and effective features. so we chose a cut point higher than important/effective features.

In [ ]:
### as you see we have a huge number of missing values. in usual case we should contact with 
#data set owner for better deciding. but in thid case we chose to drop feauturse with the 
#critical missing values. in orther we get help from article review to
#NOT remove important and effective features
miss = miss[miss['MissValues'] < 211]#extract features with less than 384 missing values
#Considering no damage to important features
final_featurs = miss.T.columns #convert the data frame rows to columns and make features list
df6 = df5[final_featurs]#extract chosen features
df6 #secound look!

We have 22 featurs! this amount of features is a double-edged sword! it can improve the accuracy but also increase the risk of overfitting. so we should extract the important features and reduce them by some method to minimize overfitting risk.

**FEATURES SELECTION AND REDUCTION**
In this level we should select features. first we used visualization and plots to select more **correlated features**. then used a machine learning algorithm called **Principal component analysis (PCA)** to reduced features. The existence of many features and little data has the risk of overfitting.

In [ ]:
#lets look closer at features

**Examining the Relationship between Features and Target**

it is useful to be able to visualize how each feature affects the diagnosis. So let’s plot a histogram for each feature and then differentiate the cognitive progression using color:

In [ ]:
# make a list of features
df_fn = df6.drop(['target'],axis = 1)
feature_names = df_fn.columns.T
feature_names

In [ ]:
#analysis the target feature 
print(df6.target) 
print(feature_names) 
print(np.array(np.unique(df6.target, return_counts=True)))

In [ ]:
import matplotlib.pyplot as plt
_,axes = plt.subplots(5,5, figsize=(15, 15))
HCL = df6[df6.target=='HCL'].to_numpy()
mMCI = df6[df6.target=='mMCI'].to_numpy()
sMCI = df6[df6.target=='sMCI'].to_numpy()
pMCI = df6[df6.target=='pMCI'].to_numpy()
AD = df6[df6.target=='AD'].to_numpy()


ax = axes.ravel()                     # flatten the 2D arrayfor i in range(22):          
for i in range(24):   # for each of the 30 features
    bins = 40    #---plot histogram for each feature---
    ax[i].hist(HCL[:,i], bins=bins, color='g', alpha=.5)
    ax[i].hist(mMCI[:,i], bins=bins, color='b', alpha=0.3) #---set the title---
    ax[i].hist(sMCI[:,i], bins=bins, color='y', alpha=.5)
    ax[i].hist(pMCI[:,i], bins=bins, color='m', alpha=0.3)
    ax[i].hist(AD[:,i], bins=bins, color='r', alpha=0.3)
    ax[i].set_title(feature_names[i], fontsize=12)        #---display the legend---
    ax[i].legend(['HCL','mMCI','sMCI','pMCI','AD'], loc='best', fontsize=8)
    
plt.tight_layout()
plt.show()

Based on this plots:

1-we have imbalance target features label(that should be consider for later in model training)

2-we have un-normalized distribution

3-w\seems we have no NOISE!

4-for each feature, if two histograms are separate, this means that the feature is important and it directly affects the target (diagnosis). (e.g. mpacctrailsb_bl make a better sample distribute sepration by target labes) 

**feature selection**:
by this information we can do feature selection. 

In [ ]:
df6_s = df6[['midtemp_bl', 'fusiform_bl', 'entorhinal_bl', 'hippocampus_bl',
       'ventricles_bl', 'imageuid_bl',
       'trabscor_bl', 'faq_bl', 'adas13_bl', 'ravlt_learning_bl', 'ravlt_immediate_bl',
       'adas11_bl', 'mpacctrailsb_bl', 'mpaccdigit_bl',
       'ldeltotal_bl', 'mmse_bl', 'adasq4_bl', 'cdrsb_bl', 'ravlt_perc_forgetting_bl','target']]

**Method 1 — Training the Model using all the Features**:

Before we perform PCA on the dataset, let’s use some classifiers to train a model using all features in the dataset and see how well it performs:

In [ ]:
#convert target numeric values
df6_s['target'].replace(['HCL', 'mMCI','sMCI','pMCI','AD'],
                        [1, 2, 3, 4, 5], inplace=True)

In [ ]:
#remove missingvalues
#NOTE: we usully need to use better method(contact with data owner, used refill method,...)
#but in this case seems drop all missing values doesnt compromised data.
df7 = df6_s.dropna()
df7.reset_index(drop=True, inplace=True)

In [ ]:
X = df7.drop(['target'],axis = 1)#features
y = df7['target']#target

In [ ]:
from sklearn.model_selection import train_test_split
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Define the model.
nb_model = GaussianNB()

# fit your model
nb_model.fit(train_X, train_y.ravel())
y_pred = nb_model.predict(val_X)

#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Define the model.
rf_model = RandomForestClassifier(n_estimators=100)
# fit your model
rf_model.fit(train_X,train_y)
val_predictions = rf_model.predict(val_X)
y_pred = rf_model.predict(val_X)


#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


**Method 2 — Training the Model using Reduced Features:**

For the next method, let’s examine the various features and try to eliminate those features that are least correlated to the target. At the same time, we also want to remove those features that exhibit muli-collinearity. The aim is to reduce the number of features and see if the accuracy of the model can be improved.

Getting the Correlation Factors:

Let’s first get the correlation of each feature with respect to the target:

In [ ]:
df_corr = df7.corr()['target'].abs().sort_values(ascending=False)
df_corr

We then extract all those features that have relatively high correlation to the target (we arbitrarily set the threshold to 0.6):

In [ ]:
# get all the features that has at least 0.6 in correlation to the 
# target
features = df_corr[df_corr > 0.6].index.to_list()[1:]
features                          # without the 'target' column

**Checking for MultiCollinearity**

Let’s remove those features that exhibits multi-collinearity:

In [ ]:
from sklearn.linear_model import LinearRegression

def calculate_vif(df7, features):   
    
    vif, tolerance = {}, {}    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df7[X], df7[feature]        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)                
        
        # calculate tolerance
        tolerance[feature] = 1 - r2        # calculate VIF
        vif[feature] = 1/(tolerance[feature])
    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

calculate_vif(df7,features)


our aim would be to remove those features that have VIF greater than 5. we can iteratively call the calculate_vif() function with different features until we have a feature-set that has all VIF values lesser than 5.

In [ ]:
# try to reduce those feature that has high VIF until each feature 
# has VIF less than 5
features = [
    'cdrsb_bl',
 'ldeltotal_bl',
 'adas13_bl'
]

calculate_vif(df7,features)

Training the Model

With the 22 features reduced to 3, let’s now train the model

In [ ]:
X = df7[[
    'cdrsb_bl',
 'ldeltotal_bl',
 'adas13_bl'
]]
y = df7['target']


In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Define the model.
nb_model = GaussianNB()

# fit your model
nb_model.fit(train_X, train_y.ravel())
y_pred = nb_model.predict(val_X)

#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


In [ ]:

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Define the model. Set random_state to 1
rf_model = RandomForestClassifier(n_estimators=100)
# fit your model
rf_model.fit(train_X,train_y)
val_predictions = rf_model.predict(val_X)

y_pred = rf_model.predict(val_X)
#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


we can see that the Naive Bayes Model improved but the Random Forrest not!

**Principal Component Analysis (PCA):**

In short, PCA is a dimensionality reduction technique that transforms a set of features in a dataset into a smaller number of features called principal components while at the same time trying to retain as much information in the original dataset as possible.

**Method 3 — Training the Model using Reduced Features (PCA):**

let’s apply PCA to the dataset and see if a better model can be trained.
Performing Standard Scaling

Remember that PCA is sensitive to scaling? So the first step is to perform a standard scaling on the features:

In [ ]:
from sklearn.preprocessing import StandardScaler

# get the features and label from the original dataframe
X = df7.iloc[:,:-1]
y = df7.iloc[:,-1]

# performing standardization
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

**Applying Principal Component Analysis (PCA)**

we can now apply PCA to the features using the PCA class in the sklearn.decomposition module:

Let’s now apply PCA to find the desired number of components based on the desired explained variance, say 85%:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.85)
pca.fit(X_scaled)

print("Cumulative Variances (Percentage):")
print(np.cumsum(pca.explained_variance_ratio_ * 100))

components = len(pca.explained_variance_ratio_)
print(f'Number of components: {components}')

# Make the scree plot
plt.plot(range(1, components + 1), 
np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel("Number of components")
plt.ylabel("Explained variance (%)")

And as we can see from the chart, 9 components are needed to cover 85% of the variability in the data.

we can also find out the importance of each feature that contributes to each of the components using the components_ attribute of the pca object:

In [ ]:
pca_components = abs(pca.components_)
#print(pca_components)

In [ ]:
#print('Top 4 most important features in each component')
#print('===============================================')
#for row in range(pca_components.shape[0]):
    # get the indices of the top 4 values in each row
   # temp = np.argpartition(-(pca_components[row]), 4)
    
    # sort the indices in descending order
  #  indices = temp[np.argsort((-pca_components[row])[temp])][:4]
    
    # print the top 4 feature names
 #   print(f'Component {row}: {df7.columns[indices].to_list()}')

**Transforming all the 24 Columns to the 10 Principal Components**


In [ ]:
X_pca = pca.transform(X_scaled)
#print(X_pca.shape)
#print(X_pca)

**Creating a Machine Learning Pipeline**

Let’s now create a machine learning pipeline so that we can formalize the entire process:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics


_sc = StandardScaler()
_pca = PCA(n_components = 0.85)
_model = RandomForestClassifier(n_estimators=100)
_model2 =GaussianNB()


rf_model = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('Classifier', _model)
])

nb_model = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('Classifier', _model2)
])

In [ ]:
# perform a split
random_state = 12
train_X, val_X, train_y, val_y = \
    train_test_split(X, y, 
                     test_size=0.3,
                     shuffle=True, 
                     random_state=random_state)

In [ ]:
# fit your model
nb_model.fit(train_X, train_y.ravel())
y_pred = nb_model.predict(val_X)

#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


In [ ]:
# train the model using the PCA components
rf_model.fit(train_X,train_y)
val_predictions = rf_model.predict(val_X)

y_pred = rf_model.predict(val_X)
#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


as we can see, PCA cant help to improve neither Naive Bayes nor Random Forest.

**Improving model use other methods**

**use KFOLD for a better data selection**

In [ ]:
X = df7.drop(['target'],axis = 1)#features
y = df7['target']#target

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Define the model.
rf_model = RandomForestClassifier(n_estimators=100)
# fit your model
rf_model.fit(train_X,train_y)
val_predictions = rf_model.predict(val_X)
y_pred = rf_model.predict(val_X)


#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(20)

print(cross_val_score(rf_model,X,y.ravel(), cv=k_fold, n_jobs=1))


In [ ]:
df_n = df7.drop(df.index[0:463])
df_n = df_n.drop(df.index[528:596])
df_n = df_n.drop(df.index[990:1057])
df_n = df_n.drop(df.index[1240:1306])
df_n.reset_index(drop=True, inplace=True)


In [ ]:
X = df_n.drop(['target'],axis = 1)#features
y = df_n['target']#target

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Define the model.
nb_model = GaussianNB()

# fit your model
nb_model.fit(train_X, train_y.ravel())
y_pred = nb_model.predict(val_X)

#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Define the model.
rf_model = RandomForestClassifier(n_estimators=100)
# fit your model
rf_model.fit(train_X,train_y)
val_predictions = rf_model.predict(val_X)
y_pred = rf_model.predict(val_X)


#---evaluate the model---
print('Model Accuracy:\n', metrics.accuracy_score(val_y,y_pred))

print('confusion_matrix:')
print(metrics.confusion_matrix(val_y,y_pred))

print('f1_score:\n', metrics.f1_score(val_y,y_pred, average='macro'))


by extracting better data we improve Model accuracy up to 85.

Conclusion:
In recap: 
in this study we tried to train a Classifier Machine Learning Model to predict the dementia progression in Mild Cognitive Impairment patients and getting Alzheimer from healthy participants. We used just non-invasive features like medical imaging and cognitive test scores. After data cleanation and analysis, we traind to model, a Naive Bayes model and a Random Forest one. Then we tryed different technique to improve the models like, feature selection, feature reduction( principal component analysis(PCA)), and Kfold data extraction. Finally its seems that Random Forest Classifier is the best model with about 85 accuracy.
